<font face="微软雅黑" color=green size=5>Julia基础Ⅱ</font>

# 作用域

变量的作用域是某个变量可见的范围。 同名的变量使得问题变得复杂， 变量作用域使得同名的不同变量能够区分开来。

变量作用域都是某些程序结构的范围内， 比如一个函数定义范围，而不是任意的一段程序行的范围。

有两种主要的作用域：
全局作用域
局部作用域  
**全局作用域适用于模块(module)内，baremodule内， 或者在命令行环境内。 每个模块有自己的全局变量， 命令行运行的程序相当于main模块。**

**for, while, 自省(comprehensions)，try-catch-finally, let结构内部构成软局部作用域。 而begin复合语句，if结构，不构成局部作用域。**

硬局部作用域是函数定义，struct, macro中。

## 句法作用域

函数可以读取其外围环境中的变量的值。 这里“外围环境”的定义Julia规定为句法作用域(lexical scoping), 即一个自定义函数的作用域的外围环境是定义此函数的环境， 而不是运行时调用这个函数的环境。

例如：

In [1]:
module Amod
  x = 1
  foo() = x
end
import .Amod
x = -1
println(Amod.foo())

1


最后一个语句调用Amod.foo()时用到的是模块Amod的全局变量x， 这是定义foo()时包含函数定义的代码中的x变量。 赋值为-1的变量x是命令行的全局变量， 这是调用foo()时所处环境中的变量。

要注意的是， 函数使用所处环境中的变量的当前值， 这不一定是定义该函数时的变量值。如

In [2]:
module Amod2
  x = 1
  foo() = x
  function setx!(xnew) 
    global x = xnew
    return ()
  end
end
import .Amod2
Amod2.setx!(2)
x = -1
println(Amod2.foo())

2


以上程序中调用Amod2.foo()函数时， 其依赖的变量Amod.x的值已经被修改为2了， 所以Amod2.foo()返回值不是1而是2。

一个模块内的所有代码可以区分为不同的作用域。

不在任何函数内的变量是全局变量。
函数的自变量和函数内用local声明的变量是属于函数的自变量。
for, while, let环境内用local声明的变量是属于环境本身局部作用域的局部变量。

## 全局作用域

在命令行环境中定义的变量属于命令行环境的全局作用域，实际是Main模块的全局作用域。

每一个模块有自己的全局作用域，但是没有一个统一的全局作用域。 模块内在所有函数定义外部赋值的变量为全局变量。 模块内的任何位置用global关键字声明的变量为该模块的全局变量。

为了访问其它模块的全局变量，可以用using或者import引入， 也可以用“模块名.变量名”格式。 事实上，每个模块是一个名字空间。 只有同一模块的代码可以修改本模块的全局变量， 在模块外可以读取模块内全局变量的值但是不允许直接对模块内的全局变量赋值修改， 可以通过调用同一模块的函数间接地修改模块内的全局变量。

使用全局变量容易造成程序漏洞， 尤其是修改全局变量的值会造成难以察觉的错误。 在为全局变量赋值时用const前缀声明其为常数全局变量， 这样的全局变量不能重新绑定， 但如果其中保存mutable值的话还是可以修改保存的值的。 这种做法可以避免使用全局变量的一些错误以及性能缺陷。

## 局部作用域

许多代码块结构都会产生一个局部作用域，如函数定义、for循环等。 局部作用域都可以读写其定义环境所处作用域（称为父作用域）的变量， 但有例外：

**如果局部作用域对变量的赋值会修改全局变量， 这时如果不在局部作用域中用global声明该变量， 程序出错；
如果在局部作用域中用local声明了变量， 则对该变量的修改不会影响父作用域的变量。**
局部作用域不是名字空间， 所以内层可以访问外层变量， 但是外层无论如何不能访问内层作用域的局部变量。

按照对父作用域的变量如何继承来区分， 局部作用域分成硬局部作用域和软局部作用域。

下面举例说明这些作用域规则。 假设每个例子都是在重新启动REPL的命令行环境内执行， 这样没有其它全局变量的干扰。

### 例1

下面的例子说明， 局部作用域独有的变量在父作用域内无法访问。 例子中for循环内构成了一个局部作用域， 其中独有的变量z无法在其父作用域即f1()函数局部作用域内访问。

In [3]:
function f1()
  for i=1:5
    z = i
  end
  println(z) # 错误：z无定义
end

f1 (generic function with 1 method)

如果调用f1()，会出错：

UndefVarError: z not defined
即在for循环外部变量z无定义。

### 例2

在下面的程序中， f2()函数内构成一个局部作用域。 f2()内定义的变量z不能在外部访问。

In [4]:
function f2()
  z = 1
  println("Inside f2(): z=$z")
end

f2 (generic function with 1 method)

如果执行f2()，结果为

Inside f2(): z=1
如果执行println("Outside f2(): z=$z")，结果为

UndefVarError: z not defined

### 例3

下面的例子修改了父作用域中的变量， 变量不是全局变量也没有用local声明。 函数f3()中的for循环构成一个局部作用域， 其父作用域是f3()的局部作用域， for()循环中可以直接读取并修改父作用域中非全局的变量z的值：

In [5]:
function f3()
  z = 0
  for i=1:5
    z += i
  end
  println(z) # 15
end
f3()

15


注意for作用域中z不是全局变量也没有用local声明。

### 例4

在f3()定义中，如果在for结构内用local声明变量z， 则程序会出错。

下面的例子说明f4()中的局部变量z与其中的for结构中的z是两个不同的变量， 因为在for结构中用local声明了该结构中的z是局部版本：

In [6]:
function f4()
  z = 0
  for i=1:3
    local z
    z = i
    println("i=$i z=$z")
  end
  println("Outside for loop: z=$z") # 0
end
f4()

i=1 z=1
i=2 z=2
i=3 z=3
Outside for loop: z=0


### 例5

In [7]:
z = 0
for i=1:5
  z += i
end

程序结果为：

ERROR: UndefVarError: z not defined
在Jupyter中运行时可能会因为修改了规则而不报错。

这个问题的正确做法是将程序像f3()那样写在一个函数中， 这就不会发生在局部作用域内修改全局变量的问题。 在局部作用域用global声明要修改的全局变量也可以：

In [8]:
z = 0
for i=1:5
  global z
  z += i
end
println(z) # 15

15


使用global关键字要慎重， 一旦在局部作用域中将某个变量用global声明为全局变量， 则此变量就不仅仅可以被其父作用域访问， 而是在模块内全局可访问。 如

In [9]:
function f5()
  for i=1:5
    global z=i
  end
  println("Outside for structure: z=$z") 
end
f5()
println("Outside function f5(): z=$z")

Outside for structure: z=5
Outside function f5(): z=5


## 软局部作用域

软局部作用域内的变量默认是在其父作用域内的变量， 但是:

在软局部作用域内新定义的变量，作用域外仍不能访问;
软局部作用域内用local声明过的变量仅能在该作用域内访问而不会与其父作用域的同名变量冲突;
软局部作用域内试图为全局变量赋值而未在作用域内用global声明该变量会出错。
for循环，while循环，自省结构， try-catch-finally结构，let结构会引入软局部作用域。 软局部作用域，如for循环， 一般用来处理其父作用域（一般是函数内部）内的变量， 与其周围代码是不可分割的。 比如在用for循环做累加时， 累加结果变量一定是在for循环父作用域内而不能是局部的, 所以软作用域非以上三种特殊情况下可以读写其父作用域的变量。

for循环和自省结构中的循环变量都是结构内的局部变量， 即使在其父作用域中有同名变量也是如此， while循环没有语法上的循环变量所以不受此限制。 在结构中新定义的变量都是结构内的局部变量， 但如果父作用域是函数的局部作用域且父作用域内有同名变量， 则结构内的变量读写父作用域中的变量。

软局部作用域的这些规定与例外规定与其它程序语言存在较大差别， 初学者很容易出错。建议如下：

对软局部作用域，尽量不要在其中新定义变量;
如果新定义变量， 用local声明使其作用域变得明显可见就不会发生误读误判， 即使父作用域中没有同名变量也加上这个声明可以使得程序的意图更清楚；
对于函数内的软局部作用域，其父作用域是函数的局部作用域， 为了能够访问函数的局部变量， 软局部作用域内不需要也不应该使用global声明该变量， 因为其变量除了新定义的，都是函数的自变量和局部变量， 使用global声明的副作用是该变量成为全局变量， 而不仅仅是父作用域中可访问的变量。

## 硬局部作用域

函数定义，struct结构，宏定义内部为硬局部作用域。 其中函数定义可以嵌套在另一个函数定义中， 而struct结构和宏定义则只能在全局作用域中定义。

在硬局部作用域中，几乎所有变量都是从其父作用域内继承来的， 例外情况包括：

对父作用域内的变量赋值，会修改全局变量时， 这时赋值会产生一个局部副本而不修改全局变量值。 注意软局部作用域没有用global声明而修改全局变量会出错， 而不是默默地生成一个局部副本。
用local声明的变量，仅在此局部作用域内起作用， 即使父作用域中有同名变量或者有同名的全局变量。
在硬局部作用域中， 父作用域中的变量以及全局变量都可以不经声明直接读取值； 父作用域中的变量如果不是全局变量， 可以直接修改变量值。

硬局部作用域中不经声明不能修改全局变量值。 需要在局部作用域内用global声明该变量才能修改全局变量值。 为了能明显地反映程序意图， 在局部作用域内不论读或者写访问全局变量时， 都最好在局部作用域内用global关键字声明该变量。

### 例6

例如， 在函数内部读取全局变量的值， 可以不用global声明：

In [10]:
z = -1
function f6()
  println("函数内读取外部变量：z=$z")
end
f6()

函数内读取外部变量：z=-1


但是， 在函数内没有用global声明的全局变量， 不能修改， 意图修改全局变量的代码实际是建立了一个局部变量：

In [11]:
z = -1
function f7()
  z = 1
  println("函数内不用global声明修改全局变量，修改后：z=$z")
end
f7()
println("退出函数后全局变量：z=$z")

函数内不用global声明修改全局变量，修改后：z=1
退出函数后全局变量：z=-1


这说明函数f7()内并没有修改全局变量z的值， f7()运行期间显示的z是一个局部变量。 函数内并没有能够修改外部的z变量值， 而且一旦函数内给z赋值，整个函数体内z都是局部变量， 这样在给z赋值之前z是无定义的， 而不是能访问外部的z值。 下面的程序在z=1赋值之前显示z的值， 这时的z已经是局部变量，所以程序会出错：

In [12]:
z = -1
function f7()
  println("函数内不用global声明修改全局变量，修改前：z=$z")
  z = 1
  println("函数内不用global声明修改全局变量，修改后：z=$z")
end
f7()
println("退出函数后全局变量：z=$z")

LoadError: UndefVarError: z not defined

所以，函数内赋值的变量， 最好用local声明以避免误解。 嵌套定义的函数是一个例外， 嵌套定义的函数的父作用域是另一个函数的局部作用域， 不存在修改全局变量的问题。

在函数内用global声明变量， 就可以读写访问全局变量。 为了程序意图更清晰， 即使仅读取全局变量， 最好也用global声明。 如

In [13]:
z = -1
function f8()
  global z
  z = 1
  println("函数内用global声明修改全局变量：z=$z")
end
f8()
println("退出函数后全局变量：z=$z")

函数内用global声明修改全局变量：z=1
退出函数后全局变量：z=1


上述程序中所有的变量z都是全局变量z。

## 嵌套定义函数的作用域

嵌套地定义在函数内的函数， 其作用域与直接定义在全局作用域的函数不同：

直接定义在全局作用域的函数的父作用域是全局作用域， 父作用域的变量是全局变量， 按照规定， 函数内修改没有用global声明的变量只能生成一个同名局部变量；
嵌套地定义的函数， 其父作用域是另一个函数的局部作用域， 所以在嵌套定义内可以不需要声明直接读写父作用域中的变量， 实际上也不能用global声明父作用域中的变量。
对于struct结构和宏定义， 它们只能在全局作用域定义而不能嵌套定义， 所以不存在这个差别。

### 例7

In [14]:
x = "global.x" # 全局变量
function foo()
  local x = "foo.x" # 这是一个baz作用域内的局部变量
  function bar()
    println("在bar()开始时: x=$x") # baz.x，读取父作用域的局部变量
    x = "bar.x" # 内嵌函数内，允许读写访问其所在函数的局部变量
    println("在bar()修改后: x=$x") # bar.x，读取父作用域的局部变量
  end
  bar()
  println("在bar()结束后: x=$x") # bar.x，函数baz()的局部变量x被内嵌函数修改了
end
foo()
println("在foo()结束后: x=$x") # global.x, 内嵌函数没有修改全局变量

在bar()开始时: x=foo.x
在bar()修改后: x=bar.x
在bar()结束后: x=bar.x
在foo()结束后: x=global.x


可以看到，baz()和bar()内的x都与全局的x="globa.x"没有关系。 嵌套定义的bar()内能读取其父作用域baz作用域的x的值， 也修改了其父作用域中的x的值， 这一点与非内嵌函数修改全局变量不同。

### 例8

之所以规定嵌套函数可以直接读写访问其父作用域中的变量， 是实现所谓“闭包”(closure)的要求。 闭包是带有状态的函数， 因为Julia不支持如Java、C++、Python这些语言的“类”(class)， 所以需要利用闭包来实现记忆状态的函数。 在这一点上Julia语言与R语言比较相近。

在下面的例子中，f9()是一个局部作用域， 在f9()内嵌套地定义了无名函数并且将其作为函数f9()的返回值, 这时f9()的函数值是一个函数对象。 通过将f9()的函数值赋值给变量counter, 变量counter就变成了一个函数， 按照句法作用域规则， counter()函数可以完全读写访问其定义时的父作用域f9()中的变量state, state就变成了counter()函数的私有状态变量， 可以保存上次运行状态：

In [15]:
function f9()
  state = 0
  function ()
    state += 1
  end
end
counter = f9()
println(counter(), ", ", counter())

1, 2


## 硬作用域和软作用域的比较

软局部作用域，如for循环， 一般用来处理其父作用域（一般是函数局部作用域）内的变量， 与其周围代码是不可分割的。 比如在用for循环做累加时，累加结果变量一定是在for循环父作用域内而不能是局部的, 所以软作用域可以读写其父作用域的变量。

另一方面，硬作用域一般是独立地运行在不同的调用场合的， 与周围的代码的关系没有那么紧密，很可能会在别的模块中被调用。 所以硬作用域不允许修改全局变量值（除非使用global声明）。

# 数据类型

**Julia中没有class，也没有子类的继承关系，所有具体类型都是最终的，并且只有抽象类型可以作为超类型，Julia中的继承是继承行为，而不是继承结构，即继承类型，不继承类。**

程序中的常量和变量都有类型，比如，常数108的类型为Int64。函数**typeof()** 可以返回常量或变量的类型，如

In [16]:
typeof(108)

Int64

变量的类型由其中保存的值的类型决定，Julia变量实际是“绑定”到了某个保存了值的地址。如

In [17]:
x = 108
typeof(x)

Int64

**函数typemax()可以求一个数值类型能保存的最大值， typemin()可以求一个数值类型能保存的最小值**。如

In [18]:
show([typemin(Int8), typemax(Int8)])

Int8[-128, 127]

Julia的变量不必须声明类型， 但是必须初始化，未定义的变量用于计算会出错。

Julia归类到动态类型语言， 是因为不必须声明变量类型。 其它的动态类型语言如Python不支持声明变量类型， Julia是允许不声明变量类型， 也可以声明变量类型， 声明变量类型往往可以改善程序效率， 尤其是函数的自变量类型。 同名的函数可以因为其自变量类型声明的不同而执行不同的操作， 这称为多重分派(multiple dispatch)。

在循环内部声明变量类型可以避免因为每次查询变量类型而引起的运行效率损失。

## 类型系统

用程序语言术语描述， Julia语言的类型系统是动态的，主格的(nominative), 参数化的。 通用类型可以带有类型参数。 类型的层次结构是显式声明的， 不是由兼容结构隐含的。 实体类型(concrete types)不能互相继承， 这与很多面向对象系统不同。 Julia的理念认为继承数据结构并不重要，继承行为才是最有用的。

Julia的数值类型可以由如下的类型结构组成：

**所有的数值都是Number， Number又分为实数(Real)和其它数，实数包括Integer和AbstractFloat，Integer中有Signed和Unsigned，Signed中有Int8, Int16, Int32, Int64, Int128等，Unsigned中有UInt8, UInt16, UInt32, UInt64, UInt128等。AbstractFloat中有Float16, Float32, Float64等，布尔类型Bool是Integer的子类**。

整数类型默认为Int64，浮点数类型默认为Float64。程序中如1.2和1.2e-3这样的字面浮点数的类型是Float64， 单精度数(Float32)可以写成如1.2f0, 1.2f-3这样的格式。BigInt是任意精度整数， BigFloat是任意精度浮点数。 实际上，BigFloat有一个用户可修改的的有效位数设置。

**用$<:$运算符来判断子类关系是否成立**，如

In [19]:
Int32 <: Number

true

In [20]:
Int32 <: AbstractFloat

false

**确定 x 是否属于给定类型。也可以用作中缀运算符**，例如x isa 类型。

In [36]:
isa(1, Int)

true

In [37]:
isa(1, Matrix)

false

In [38]:
1 isa Number

true

## 复数类型

Julia内建了复数类型， 这是Number的子类。 **关键字im用来表示复数虚部**， 比如1.0 + 2.1im表示复数 1+2.1i。

复数类型用实数类型分别保存实部和虚部， 其定义利用了Julia语言的“参数化类型”： 实部与虚部的实际类型可以是一个“类型参数”。 比如， Complex{Float64}用Float64(即双精度实数)分别保存实部和虚部。

对复数类型可以应用函数abs(), exp(), sqrt(), real(), imag()等。 如

In [21]:
abs(1.0 + 2.1im)

2.3259406699226015

In [22]:
sqrt(1.0^2 + 2.1^2)

2.3259406699226015

## 有理数类型

Julia内建了有理数类型， 分别保存分子和分母， Rational{Int64}用Int64类型分别保存分子和分母。 有理数是Number的子类， 支持四则运算， 也可以与其他数值类型混合进行四则运算。

用2 // 5表示有理数 2/5 ， 其数据类型为Rational{Int64}。 可以用函数num()取出分子， 用denom()取出分母， 用float()转换为浮点实数。

函数内部以及某些控制结构如for循环结构内， 定义的变量是局部的， 全局变量在局部作用域内的访问有特殊规则。 函数内部可以嵌套定义函数， 嵌套在内部的函数的局部变量又有特殊的作用域规则。

## 类型转换与提升

设T为类型名， T(x)在可以无损转换的情况下将自变量x转换为T类型返回， 但是不能无损转换时会报错。如

In [23]:
Int64(1.0)

1

In [24]:
Int64(1.5)

LoadError: InexactError: Int64(1.5)

In [25]:
Float64(1)

1.0

In [26]:
Int64(true)

1

In [27]:
Bool(1)

true

**string()可以将大多数类型转换为字符型， 也可以连接多项**，如

In [28]:
string(1+2)

"3"

In [29]:
string("1+2=", 1+2)

"1+2=3"

反过来，**为了将字符串中的数字转换成数值型， 用parse()函数**， 如

In [30]:
parse(Float64, "1.23") + 10

11.23

通过多重派发定义各个运算符， 使得Julia的四则运算、比较运算等表达式中可以混合使用布尔型、整型、浮点型数， 参与运算的数据类型自动提升到更一般的数据型。 如

In [31]:
1 + 2*1.5 - false

4.0

In [32]:
convert(Float64, 128) #将128转为Float64类型

128.0

## 类型声明

**Julia用$::$表示类型归属**， 此运算符有两个含义：类型验证(assertion)与类型声明。

### 类型验证

**在变量名和表达式末尾添加::然后可以要求类型验证。 如x::Float64，(x+1)::Int64**。 这样做的理由是

作为一个额外的验证确保程序是按照预想的逻辑执行的，
给编译器提供类型信息以改善程序效率。
::符号的意思是“is instance of”(是某某类的一个实例)， 即左边的表达式是右边的类的一个实例 如果类型是实类型，则左边必须确实是此类型； 如果类型是抽象类型，则左边应该是其子类。 类型声明不成立的时候会发生异常，否则结果是左边的值。

例如，

In [33]:
(1+2)::AbstractFloat

LoadError: TypeError: in typeassert, expected AbstractFloat, got a value of type Int64

In [34]:
(1+2)::Int

3

**类型验证无误，则验证不起作用**， 结果为表达式的值3。

### 类型声明

局部变量类型声明例如

In [30]:
function foo1()
  local x::Int8
  x = 100
  x
end
typeof(foo1())

Int8

In [36]:
function sqp(x)::Float64
    return sqrt(x+1)
end
[typeof(sqp(0)), typeof(sqp(1))]

2-element Vector{DataType}:
 Float64
 Float64

通过对函数返回类型的声明， 不论结果是 1 还是 $\scriptsize \sqrt{2}$ ， 结果类型都是Float64。

## 抽象类型

**抽象类型不能实例化， 只是作为实类型的父类**， 好处是可以使得函数自变量类型取一类类型。 比如将函数自变量声明为Number， 则输入的值为Int8, Float64等都是允许的。 

抽象类型包括：Number，Real，AbstractFloat，Integer，Signed，Unsigned  
实类型包括：Int8, Int16, Int32, Int64, Int128，UInt8, UInt16, UInt32, UInt64, UInt128，Float16, Float32, Float64

### 类型共用体Union

In [8]:
IntorString = Union{Int, AbstractString} #声明一个类型共用体，里面有两种类型构成。

Union{Int64, AbstractString}

In [9]:
123::IntorString

123

In [10]:
"abc"::IntorString

"abc"

In [11]:
12.3::IntorString #12.3不属于这两种类型中的任一个

LoadError: TypeError: in typeassert, expected Union{Int64, AbstractString}, got a value of type Float64

In [12]:
f(x::IntorString) = println(x) #函数中声明了x的类型

f (generic function with 1 method)

In [13]:
f(123)

123


In [14]:
f(12.3)

LoadError: MethodError: no method matching f(::Float64)
[0mClosest candidates are:
[0m  f([91m::Union{Int64, AbstractString}[39m) at In[12]:1

## 初等类型

初等类型就是基础的二进制表示，如整数，浮点数。 Julia允许自定义初等类型，用primitive type命令。 Julia提供的初等类型也是用Julia定义的。 详见Julia手册。

## 复合类型

**用struct定义的复合类型是不可修改的(immutable)**，这样的好处是

更高效。有时可以高效地包装进数组中，有时甚至不需为其分配额外存储空间。
不能修改构造器提供的值。
不能修改的内容，也使得程序比较简单。
如果某个域是可修改类型(mutable)，如数组， 则该域保存的内容还是可以修改的， 这里不可修改是指这样的域不能再绑定到别的对象上。

In [13]:
struct Foo
    x1
    x2::Int
    x3::Float64
end
foo = Foo("Hello", 10, 11.9)
typeof(foo)

Foo

In [11]:
typeof(Foo)

Core.SimpleVector

In [15]:
typeof(foo.x2)

Int64

In [5]:
foo.x2 = 2

LoadError: setfield!: immutable struct of type Foo cannot be changed

**用mutable struct定义可修改复合类型(Mutable composite types)， 其域的值可以修改**。 这样的数据类型一般在堆上分配内存，有稳定的内存地址。 这是一个容器，不同运行时刻的容器内容可以变化， 但是其地址不变。 另一方面，不可变的类型是与各个域的值紧密联系的， 域的值就决定了对象的一切。

在赋值和函数参数传递时不可变类型复制传递， 可变类型按引用传递； 不可变复合类型的域不可修改 （不能绑定到其它值，但是如果域本身是可变类型还是可以修改内容的）。 典型的可变类型是数组， 而元组(tuple)是不可变类型。

In [25]:
mutable struct Foo2
    x1
    x2::Int
    x3::Float64
end
foo = Foo2("Hello", 10, 11.9)
foo.x2 = 2

2

## 参数类型

参数类型是类似于C++中模板(template)的类型。 **一个类型可以将其元素的值类型作为类型参数， 一次性地定义一批类型**。 参数类型使得同一算法可以用来处理不同类型的数据。

数组(Array)就是参数类型。 Vector{Float64}或Array{Float64,1} 就是元素为Float64的一维数组， 而Vector{String}或Array{String,1} 就是字符串的一维数组。 Array{Float64,2}是元素为Float64的二维数组， 也称为矩阵。 这里Float64, String就是参数类型的类型参数。

### 复合参数类型

In [16]:
#在参数定义时，指定类型T
struct Pos{T}
    x::T
    y::T
end         

In [17]:
a = Pos{Int64}(1, 2)
b = Pos{Float64}(1.1, 2.1)
c = (1, 2)
d = (1.1, 2.1)
e = (1, 2.1)

(1.1, 2.1)

In [18]:
typeof(c)

Tuple{Int64, Int64}

In [19]:
typeof(d)

Tuple{Float64, Float64}

In [22]:
typeof(e)

Tuple{Int64, Float64}

每个实例都是Pos的子类型。

In [23]:
Pos{Float64} <: Pos

true

但不同的T声明的具体类型之间不能互为子类型。

In [24]:
Pos{Float64} <: Pos{Real} #虽然Float64 <: Real，但Pos{Float64} <: Pos{Real}不行

false

### 抽象参数类型

由抽象类型而来，就是给抽象类型加了个参数

In [25]:
abstract type Pointy{T} end

In [26]:
Pointy{Int64} <: Pointy

true

In [28]:
Pointy{Int64} <: Pointy{Real} #不同T之间不能互为类型

false

### 原始参数类型

由原始类型而来，就是给原始类型加了个参数

In [32]:
primitive type Pri{T} 64 end

In [33]:
Pri{Float64} <: Pri

true

### 元组类型

In [34]:
t = tuple(1,2,3)

(1, 2, 3)

## UnionAll类型

UnionAll是某些参数的所有值的类型的迭代并集，通常使用关键字**where**编写。

In [64]:
struct Pos{T}
    x::T
    y::T
end   

function f1(p::Pos{T} where T<:Real)
    p
end

f1(Pos{Int64}(1,2))

Pos{Int64}(1, 2)

当有两个类型参数时

In [65]:
struct Pos2{T1, T2}
    x::T1
    y::T2
end   

function f2(p::Pos2{T1, T2} where T1<:Int64 where T2<:Float64)
    p
end

f2(Pos2{Int64, Float64}(1, 1.1))

Pos2{Int64, Float64}(1, 1.1)

where还可以限定上下界

In [69]:
function f3(p::Pos{T} where Int64 <: T <:Real)
    p
end

f3(Pos{Int64}(1,2))

Pos{Int64}(1, 2)

# 方法

**Julia的函数可以通过给自变量声明类型实现高效代码， 同时也可以使得程序意图更明显。 同一个函数可以有不同类型的自变量， 这其实是多个函数共用同一个函数名， 称这些函数为该函数名的“方法”（methods)**。 用类似于C++模板的方法可以将函数自变量类型参数化， 这样同一算法可以适用于多个自变量类型。

这部分技术比较复杂， 不恰当地应用可以造成程序错误， 初学者可以暂时忽略， 自定义函数时先不声明自变量类型。 等发现某个函数造成了运行效率瓶颈， 或者Julia语言已经运用比较纯熟时， 再改进原来的函数， 使其对具体的不同自变量类型有不同的实现。

In [1]:
methods(+) #查看函数里面的方法，这里查看的是函数+里面的方法

# 208 methods for generic function "+":
[1] +(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:87
[2] +(c::Union{UInt16, UInt32, UInt8}, x::BigInt) in Base.GMP at gmp.jl:529
[3] +(c::Union{Int16, Int32, Int8}, x::BigInt) in Base.GMP at gmp.jl:535
[4] +(c::Union{UInt16, UInt32, UInt8}, x::BigFloat) in Base.MPFR at mpfr.jl:390
[5] +(c::Union{Int16, Int32, Int8}, x::BigFloat) in Base.MPFR at mpfr.jl:398
[6] +(c::Union{Float16, Float32, Float64}, x::BigFloat) in Base.MPFR at mpfr.jl:406
[7] +(x::Union{Dates.CompoundPeriod, Dates.Period}) in Dates at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\Dates\src\periods.jl:372
[8] +(A::LinearAlgebra.Tridiagonal, B::LinearAlgebra.Tridiagonal) in LinearAlgebra at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\tridiag.jl:740
[9] +(x::LinearAlgebra.Tridiagonal, H::LinearAlgebra.UpperHessenberg) in LinearAlgebra at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\hessenberg.jl:107
[10] +(A::LinearAlgebra.Tridiagonal, B::LinearAlgebra.SymTridiagonal) in LinearAlgebra at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\special.jl:157
[11] +(A::LinearAlgebra.Tridiagonal, B::LinearAlgebra.Diagonal) in LinearAlgebra at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\special.jl:173
[12] +(A::LinearAlgebra.Tridiagonal, B::LinearAlgebra.Bidiagonal) in LinearAlgebra at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\special.jl:193
[13] +(A::LinearAlgebra.Tridiagonal{var"#s859", V} where {var"#s859"<:Number, V<:AbstractVector{var"#s859"}}, B::LinearAlgebra.UniformScaling) in LinearAlgebra at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\special.jl:226
[14] +(z::Complex, w::Complex) in Base at complex.jl:288
[15] +(A::SparseArrays.AbstractSparseMatrixCSC, B::SparseArrays.AbstractSparseMatrixCSC) in SparseArrays at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\SparseArrays\src\sparsematrix.jl:1771
[16] +(A::SparseArrays.AbstractSparseMatrixCSC, B::Array) in SparseArrays at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\SparseArrays\src\sparsematrix.jl:1774
[17] +(A::SparseArrays.AbstractSparseMatrixCSC, J::LinearAlgebra.UniformScaling) in SparseArrays at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\SparseArrays\src\sparsematrix.jl:3760
[18] +(F::LinearAlgebra.Hessenberg, J::LinearAlgebra.UniformScaling) in LinearAlgebra at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\hessenberg.jl:640
[19] +(Da::LinearAlgebra.Diagonal, Db::LinearAlgebra.Diagonal) in LinearAlgebra at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\diagonal.jl:172
[20] +(D::LinearAlgebra.Diagonal, S::LinearAlgebra.Symmetric) in LinearAlgebra at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\diagonal.jl:176
[21] +(D::LinearAlgebra.Diagonal{var"#s859", V} where {var"#s859"<:Real, V<:AbstractVector{var"#s859"}}, H::LinearAlgebra.Hermitian) in LinearAlgebra at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\diagonal.jl:182
[22] +(x::LinearAlgebra.Diagonal, H::LinearAlgebra.UpperHessenberg) in LinearAlgebra at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\hessenberg.jl:107
[23] +(A::LinearAlgebra.Diagonal, B::LinearAlgebra.Bidiagonal) in LinearAlgebra at C:\Program Files\MWorks.Syslab 2022\Tools\julia-1.7.1\share\julia\stdlib\v1.7\LinearAlgebra\src\special.jl:125
[24] +(A::LinearAlgebra.Diagonal, B::LinearAlgebra.SymTridiagon

## 多重派发

函数可以将0到多个自变量（参数）集合映射为一个返回值， 类似的操作对于不同的参数类型，可能有不同的具体实现。 比如，两个整数相加，两个浮点数相加，一个整数加一个浮点数，是类似的运算， 但实际计算过程很不一样。 尽管如此，因为其概念上的相近似，Julia中将所有这些运算都归结为+函数。

对于这种类似的操作， Julia在定义函数时， 可以不是一下就完整定义， 而是针对不同参数类型逐步地完善函数定义。 函数针对一种参数类型组合所规定的操作叫做一个方法。 借助于参数类型的声明(annotation)， 以及参数个数， 可以对同一函数定义多个方法， 每一种参数类型组合定义一个方法。 调用一个函数时， 规定类型最详细、与输入的参数类型最匹配的方法被调用。 设计时考虑比较周到的话，虽然一个函数有多种特殊的处理方法， 其结果看起来还是可以比较一致的。

**从一个函数的多种方法选择一个方法来执行的过程称为派发(dispatch)**。 Julia在派发问题上与其它的面向对象语言由很大区别， **Julia是根据参数个数不同以及所有参数类型的不同来选择方法的， 这称为多重派发(multiple dispatch)**， 其它面向对象语言一般仅根据第一个参数派发， 而且第一个参数往往不写出来。 Julia的做法更合理，更灵活、更强大。

对于常见的数学计算， 一般用Float64来计算。 可以先定义Float64版本的函数， 然后对于一般的自变量， 可声明为Number， 转换为Float64后调用已有函数即可。

例如：

In [37]:
ff(x::Float64, y::Float64) = 2x + y
ff(x::Number, y::Number) = ff(Float64(x), Float64(y))
println(ff(1.0, 2.0))
println(ff(1, 2))
println(ff(1.0, 2))

4.0
4.0
4.0


上述函数如果还希望输入的x和y都是整数时返回整数值，只要再定义一个整数输入的方法：

In [38]:
ff(x::Integer, y::Integer) = 2*Int64(x) + Int64(y)
ff(1,2)

4

注意，为了使得上述函数可以对向量使用， 不需要单独定义方法， 而只要用加点语法即可，如

In [39]:
ff.([1.0, 2.2], [-1.0, 3.3])

2-element Vector{Float64}:
 1.0
 7.7

# 宏调用

Julia的宏(Macro)是一个非常强大的代码评估工具，使用@调用宏，宏类似于函数，但函数接受正常变量作为参数，宏接受表达式并返回修改后的表达式。

两种常用的宏的调用方式：  
@name expr1 expr2 ...  
@name(expr1, expr2, ...)  

需要注意的是：第一种用法两个参数之间是用空格隔开，且参数之间没有逗号；第二种用法的name和()之间没有空格，且参数之间有逗号隔开。

## @reexport

In [3]:
using Reexport #用于从另一个模块中重新导出一个模块，使其可以使用其中的符号、函数名

In [4]:
?Reexport 

search: Reexport @reexport CapturedException current_exceptions



No docstring found for module `Reexport`.

# Exported names

`@reexport`

# Displaying contents of readme found at `D:\TongYuan\.julia\packages\Reexport\OxbHO\README.md`

# Reexport

[![Build status](https://github.com/simonster/Reexport.jl/workflows/CI/badge.svg)](https://github.com/simonster/Reexport.jl/actions?query=workflow%3ACI+branch%3Amaster)

## Introduction

Maybe you have a module `X` that depends on module `Y` and you want `using X` to pull in all of the symbols from `Y`. Maybe you have an outer module `A` with an inner module `B`, and you want to export all of the symbols in `B` from `A`. It would be nice to have this functionality built into Julia, but we have yet to reach an agreement on what it should look like (see [JuliaLang/julia#1986](https://github.com/JuliaLang/julia/issues/1986)). This short macro is a stopgap we have a better solution.

## Usage

`@reexport using <modules>` calls `using <modules>` and also re-exports their symbols:

```julia
module Y
    ...
end

module Z
    ...
end

module X
    using Reexport
    @reexport using Y
    # all of Y's exported symbols available here
    @reexport using Z: x, y
    # Z's x and y symbols available here
end

using X
# all of Y's exported symbols and Z's x and y also available here
```

`@reexport import <module>.<name>` or `@reexport import <module>: <name>` exports `<name>` from `<module>` after importing it.

```julia
module Y
    ...
end

module Z
    ...
end

module X
    using Reexport
    @reexport import Y
    # Only `Y` itself is available here
    @reexport import Z: x, y
    # Z's x and y symbols available here
end

using X
# Y (but not its exported names) and Z's x and y are available here.
```

`@reexport module <modulename> ... end` defines `module <modulename>` and also re-exports its symbols:

```julia
module A
    using Reexport
    @reexport module B
    	...
    end
    # all of B's exported symbols available here
end

using A
# all of B's exported symbols available here
```

`@reexport @another_macro <import or using expression>` first expands `@another_macro` on the expression, making `@reexport` with other macros.

`@reexport begin ... end` will apply the reexport macro to every expression in the block.


## 常见用法@

In [73]:
using MLJ #机器学习宏常见用法
@load PCA pkg=MultivariateStats verbosity=0 install=true #pkg包的名字，verbosity静默模式

MLJMultivariateStatsInterface.PCA

## 常用宏

常用宏：@time、@elapsed 返回程序运行时间。

In [1]:
@time sum([1,2,3,4])

  0.000003 seconds (1 allocation: 96 bytes)


10

@show 当该宏与任何代码一起使用时，将返回表达式并计算其结果。

In [42]:
@show println("hello!")

hello!
println("hello!") = nothing


@which 当一个函数有多个方法，并希望提供一组特定参数调用其方法时，此宏非常有用。

In [74]:
function tri(n::Int64)
    3n
end

function tri(n::Float64)
    3n
end

methods(tri) #tri有两个方法

# 2 methods for generic function "tri":
[1] tri(n::Int64) in Main at In[74]:1
[2] tri(n::Float64) in Main at In[74]:5

In [75]:
println(@which tri(10.0))
println(@which tri(10))

tri(n::Float64) in Main at In[74]:5
tri(n::Int64) in Main at In[74]:1


@task 返回一个任务而不运行它。

@assert 这个宏用来判断传入表达式的值，如果是true返回nothing，否则报错。

# 元编程

## 表达式

使用Expr构造函数创建最基本的表达式类型

任何Expr类型的对象都有三个部分：  
1）符号，比如:call，冒号加一个变量值  
2）一系列参数  
3）最后结果类型  

In [3]:
sample_expr = Expr(:call, +, 10, 20) #计算10+20，call作为第一个参数，表示表达式的头部；接下来提供了+，10，20作为参数。

:((+)(10, 20))

In [4]:
eval(sample_expr) #使用eval函数计算表达式,eval接受一个Expr类型的值作为输入

30

In [5]:
sample_expr = Expr(:call, +, :x1, :y1) #在运行时使用冒号:
x1 = 10
y1 = 20
eval(sample_expr)

30

In [6]:
dump(sample_expr) #查看Expr类型的数据内容

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: + (function of type typeof(+))
    2: Symbol x1
    3: Symbol y1


In [7]:
x2 = 10
y2 = 20
e = :($x2 + $y2) #解析表达式时插入x,y
eval(e)

30

总的来说，可以通过两种方法对Expr对象进行插值  
1）在运行时使用冒号:  
2）在解析时使用$符号  

## 元编程 

元编程（英语：Metaprogramming），是指某类计算机程序的编写，这类计算机程序编写或者操纵其它程序（或者自身）作为它们的资料，或者在编译时完成部分本应在运行时完成的工作。多数情况下，与手工编写全部代码相比，程序员可以获得更高的工作效率，或者给与程序更大的灵活度去处理新的情形而无需重新编译。

编写元程序的语言称之为元语言。被操纵的程序的语言称之为"目标语言"。一门编程语言同时也是自身的元语言的能力称之为"反射"或者"自反"。

In [8]:
function foo(n::Int64)
    for i = 1:n
        println("foo")
    end
end
foo(2)

foo
foo


In [9]:
function bar1(n::Int64)
    for i = 1:n
        println("bar")
    end
end
bar1(2)

bar
bar


In [10]:
function baz(n::Int64)
    for i = 1:n
        println("baz")
    end
end
baz(2)

baz
baz


以上很多代码是重复的，使用元编程，可以动态生成代码，节约开发时间。

In [11]:
for sym in [:foo, :bar1, :baz]
    @eval function $(Symbol(string(sym)))(n::Int64)
        for i in 1:n
            println($sym)
        end
    end
end

foo(2)

foo
foo


In [12]:
x = 1
eval(x)

1

In [20]:
function math_expr(sm,op,op1)
    expr = Expr(:call, sm, op, op1)#定义表达式的操作
    return expr
end

ex = math_expr(:+,1, 2)

:(1 + 2)

In [21]:
eval(ex)

3

# 开发扩展包

[链接1](https://pkgdocs.julialang.org/v1/creating-packages/)

[链接2](https://invenia.github.io/PkgTemplates.jl/stable/user/)

[链接3](https://ke.qq.com/itdoc/julia/lx9o1jg7.html)

In [1]:
#创建扩展包
using PkgTemplates
t = Template(user="caoyuyu")
t("C:\\Users\\Administrator\\Desktop\\MyPkg")

┌ Info: Running prehooks
└ @ PkgTemplates D:\TongYuan\.julia\packages\PkgTemplates\j6Nfl\src\template.jl:130
┌ Info: Running hooks
└ @ PkgTemplates D:\TongYuan\.julia\packages\PkgTemplates\j6Nfl\src\template.jl:130
  Activating project at `C:\Users\Administrator\Desktop\MyPkg`
    Updating registry at `D:\TongYuan\.julia\registries\General.toml`
┌ Warning: The active manifest file at `C:\Users\Administrator\Desktop\MyPkg\Manifest.toml` has an old format that is being maintained.
│ To update to the new format run `Pkg.upgrade_manifest()` which will upgrade the format without re-resolving.
└ @ Pkg.Types C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.7\Pkg\src\manifest.jl:287
  No Changes to `C:\Users\Administrator\Desktop\MyPkg\Project.toml`
  No Changes to `C:\Users\Administrator\Desktop\MyPkg\Manifest.toml`
Precompiling project...
  ✓ MyPkg
  1 dependency successfully precompiled in 0 seconds
  Activating project at `D:\TongYuan\.julia\environments\v1.7`
┌ Info: Runni